In [83]:
!ls data/mimic-iv/hosp

d_hcpcs.csv.gz		 drgcodes.csv.gz	    poe.csv.gz
diagnoses_icd.csv	 emar.csv.gz		    poe_detail.csv.gz
diagnoses_icd.csv.gz	 emar_detail.csv.gz	    prescriptions.csv
d_icd_diagnoses.csv	 hcpcsevents.csv.gz	    prescriptions.csv.gz
d_icd_diagnoses.csv.gz	 labevents.csv.gz	    procedures_icd.csv
d_icd_procedures.csv.gz  microbiologyevents.csv.gz  procedures_icd.csv.gz
d_labitems.csv.gz	 pharmacy.csv.gz	    services.csv.gz


In [38]:
# !tar -xvzf data/mimic-iv/icu/icustays.csv.gz
# !gzip -dk data/mimic-iv/icu/icustays.csv.gz
!gzip -dk data/mimic-iv/hosp/prescriptions.csv.gz

In [19]:
import os
import sys
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm
tqdm.pandas()
pd.set_option('display.max_columns', None)

In [3]:
MIMIC_PATH = "data/mimic-iv"
CUSTOM_TASKS_PATH = "data/custom_tasks"
os.makedirs(CUSTOM_TASKS_PATH, exist_ok=True)

CREATE_ICU_STAYS = True

# based on this: https://github.com/Jeffreylin0925/MIMIC-III_ICU_Readmission_Analysis/blob/master/mimic3-readmission/scripts/create_readmission.py
# will need to cite it https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0218942#sec005

In [27]:
# patients = pd.read_csv(os.path.join(MIMIC_PATH, 'core/patients.csv'))
# patients.columns = [x.upper() for x in patients.columns]

# patients.ANCHOR_YEAR = pd.to_datetime(patients.ANCHOR_YEAR, format='%Y')
# patients.head()



,SUBJECT_ID,GENDER,ANCHOR_AGE,ANCHOR_YEAR,ANCHOR_YEAR_GROUP,DOD
0,10000048,F,23,2126-01-01,2008 - 2010,NaN
1,10002723,F,0,2128-01-01,2017 - 2019,NaN
2,10003939,M,0,2184-01-01,2008 - 2010,NaN
3,10004222,M,0,2161-01-01,2014 - 2016,NaN
4,10005325,F,0,2154-01-01,2011 - 2013,NaN


In [41]:
test_stays = pd.read_csv(os.path.join(MIMIC_PATH, icu_stays_dir))  # , index_col=0
test_stays.columns = [x.upper() for x in test_stays.columns]
test_stays.head()
icu_stays_simple = test_stays[['SUBJECT_ID', 'HADM_ID', 'STAY_ID']].copy()
icu_stays_simple.head()

,SUBJECT_ID,HADM_ID,STAY_ID
0,17867402,24528534,31793211
1,14435996,28960964,31983544
2,17609946,27385897,33183475
3,18966770,23483021,34131444
4,12776735,20817525,34547665


In [28]:
icu_stays_dir = 'icu/icustays.csv'
admission_dir = 'core/admissions.csv'
patients_dir = 'core/patients.csv'


def merge_stays_counts(table1, table2):
    return table1.merge(table2, how='inner', left_on=['HADM_ID'], right_on=['HADM_ID'])

def add_inhospital_mortality_to_icustays(stays):
    mortality_all = stays.DOD.notnull() | stays.DEATHTIME.notnull()
    stays['MORTALITY'] = mortality_all.astype(int)

    # in hospital mortality
    mortality = stays.DEATHTIME.notnull() & ((stays.ADMITTIME <= stays.DEATHTIME) & (stays.DISCHTIME >= stays.DEATHTIME))

    stays['MORTALITY0'] = mortality.astype(int)
    stays['MORTALITY_INHOSPITAL'] = stays['MORTALITY0']
    return stays


def add_inunit_mortality_to_icustays(stays):
    mortality = stays.DEATHTIME.notnull() & ((stays.INTIME <= stays.DEATHTIME) & (stays.OUTTIME >= stays.DEATHTIME))

    stays['MORTALITY_INUNIT'] = mortality.astype(int)
    return stays

def read_stays():
    stays = pd.read_csv(os.path.join(MIMIC_PATH, icu_stays_dir))  # , index_col=0
    stays.columns = [x.upper() for x in stays.columns]
    # needs merge to get all the fields
    admis = pd.read_csv(os.path.join(MIMIC_PATH, admission_dir))   #, index_col=0
    admis.columns = [x.upper() for x in admis.columns]
    admission_fields = ['HADM_ID', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'DISCHARGE_LOCATION', 'INSURANCE', 'MARITAL_STATUS', 'ETHNICITY']
    
    stays = stays.merge(admis[admission_fields], on='HADM_ID', how='inner')
    patients = pd.read_csv(os.path.join(MIMIC_PATH, patients_dir))  # , index_col=0
    patients.columns = [x.upper() for x in patients.columns]
    
    patients.rename(columns={'ANCHOR_YEAR':'DOB'}, inplace=True)
    
    stays = stays.merge(patients[['SUBJECT_ID', 'DOB', 'DOD', 'GENDER']], on='SUBJECT_ID', how='inner')
    
    stays.INTIME = pd.to_datetime(stays.INTIME)
    stays.OUTTIME = pd.to_datetime(stays.OUTTIME)
    stays.ADMITTIME = pd.to_datetime(stays.ADMITTIME)
    stays.DISCHTIME = pd.to_datetime(stays.DISCHTIME)
    stays.DOB = pd.to_datetime(stays.DOB, format='%Y')
    stays.DOD = pd.to_datetime(stays.DOD)
    stays.DEATHTIME = pd.to_datetime(stays.DEATHTIME)
    stays.sort_values(by=['INTIME', 'OUTTIME'], inplace=True)
    return stays
    
def get_next_intime(row, stays):
    subj_id = row['SUBJECT_ID']
    outtime = row['OUTTIME']
    later_stays = stays[(stays.SUBJECT_ID == subj_id) & (stays.INTIME > outtime)]
    if len(later_stays) == 0:
        return None
    return later_stays.sort_values(by='INTIME', ascending=True).iloc[0].INTIME



def combine(list_of_sets):
    s = []
    for s1 in list_of_sets:
        s.extend(list(s1))
    return s

    
def safe_age(row):
    try:
        return row['ADMITTIME'] - row['DOB']
    except:
        return None

if CREATE_ICU_STAYS:
    stays = read_stays()
    stays = add_inhospital_mortality_to_icustays(stays)
    stays = add_inunit_mortality_to_icustays(stays)
    #stays = stays.drop(stays[(stays.MORTALITY == 1) & (stays.MORTALITY_INHOSPITAL == 1) & (stays.MORTALITY_INUNIT == 1)].index)

    # how many icu stays per hospital admission
    counts = stays.groupby(['HADM_ID']).size().reset_index(name='COUNTS')
    stays = merge_stays_counts(stays, counts)
    # binary column: is this stay the last one in the hospital admission?
    max_outtime = stays.groupby(['HADM_ID'])['OUTTIME'].transform(max) == stays['OUTTIME']
    stays['MAX_OUTTIME'] = max_outtime.astype(int)

    # was the patient transferred back to the icu, during this admission, after this stay?    
    transferback = (stays.COUNTS > 1) & (stays.MAX_OUTTIME == 0)
    stays['TRANSFERBACK'] = transferback.astype(int)

    # Did the patient die in the hospital but out of the icu?    
    dieinward = (stays.MORTALITY == 1) & (stays.MORTALITY_INHOSPITAL == 1) & (stays.MORTALITY_INUNIT == 0)
    stays['DIEINWARD'] = dieinward.astype(int)


    # take only icu stays that were the last in their admission and calculate the time until the next admission
    # TODO: this is wrong!!! Maybe we're taking admissions of other patients??
    #next_admittime = stays[max_outtime]
    #next_admittime = next_admittime[['HADM_ID', 'ICUSTAY_ID', 'ADMITTIME', 'DISCHTIME']]
    #next_admittime['NEXT_ADMITTIME'] = next_admittime.ADMITTIME.shift(-1)
    #next_admittime['DIFF'] = next_admittime.NEXT_ADMITTIME - stays.DISCHTIME
    #stays = merge_stays_counts(stays, next_admittime[['HADM_ID', 'DIFF']])

    stays['NEXT_INTIME'] = stays.progress_apply(lambda row: get_next_intime(row, stays), axis=1)
    stays['DIFF'] = stays['NEXT_INTIME'] - stays['OUTTIME']


    less_than_30days = stays.DIFF.notnull() & (stays.DIFF < '30 days 00:00:00')
    #less_than_30days = stays.DIFF.notnull() & (stays.DIFF < 30)
    stays['LESS_THAN_30DAYS'] = less_than_30days.astype(int)

    # did the patient die after being discharged? (from the hospital, not from the ICU)
    #stays['DISCHARGE_DIE'] = (stays.DOD - stays.DISCHTIME).apply(lambda s: s / np.timedelta64(1, 's')) / 60./60/24
    stays['DISCHARGE_DIE'] = stays.DOD - stays.DISCHTIME
    stays['DIE_LESS_THAN_30DAYS'] = (stays.MORTALITY == 1) & (stays.MORTALITY_INHOSPITAL == 0) & (stays.MORTALITY_INUNIT == 0) & (stays.DISCHARGE_DIE < '30 days 00:00:00')
    stays['DIE_LESS_THAN_30DAYS'] = stays['DIE_LESS_THAN_30DAYS'].astype(int)

    # final label calculation

    stays['READMISSION'] = ((stays.TRANSFERBACK==1) | (stays.DIEINWARD==1) | (stays.LESS_THAN_30DAYS==1) | (stays.DIE_LESS_THAN_30DAYS==1)).astype(int)

    # patients who died in the ICU - mark with "2" in the READMISSION column
    stays.loc[(stays.MORTALITY == 1) & (stays.MORTALITY_INHOSPITAL == 1) & (stays.MORTALITY_INUNIT == 1), 'READMISSION'] = 2
    stays.to_csv(os.path.join(CUSTOM_TASKS_PATH, 'data', 'stays_readmission.csv'))
    print(f"{len(stays)} ICU stays. \מ{stays.READMISSION.value_counts()}") 

else:
    stays = pd.read_csv(os.path.join(CUSTOM_TASKS_PATH, 'data', 'stays_readmission.csv'), index_col=0)

    


100%|██████████| 76540/76540 [01:15<00:00, 1007.75it/s]


76540 ICU stays. \מ0    58331
1    12940
2     5269
Name: READMISSION, dtype: int64


In [29]:
stays.READMISSION.value_counts()

0    58331
1    12940
2     5269
Name: READMISSION, dtype: int64

In [37]:
display(stays)

,SUBJECT_ID,HADM_ID,STAY_ID,FIRST_CAREUNIT,LAST_CAREUNIT,INTIME,OUTTIME,LOS,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,MARITAL_STATUS,ETHNICITY,DOB,DOD,GENDER,MORTALITY,MORTALITY0,MORTALITY_INHOSPITAL,MORTALITY_INUNIT,COUNTS,MAX_OUTTIME,TRANSFERBACK,DIEINWARD,NEXT_INTIME,DIFF,LESS_THAN_30DAYS,DISCHARGE_DIE,DIE_LESS_THAN_30DAYS,READMISSION,AGE
0,18106347,24305596,30588857,Cardiac Vascular Intensive Care Unit (CVICU),Cardiac Vascular Intensive Care Unit (CVICU),2110-01-11 10:16:06,2110-01-12 17:17:47,1.292836,2110-01-11 10:14:00,2110-01-15 17:31:00,NaT,EW EMER.,PACU,HOME HEALTH CARE,Other,DIVORCED,WHITE,2110-01-01,NaT,F,0,0,0,0,1,1,0,0,NaT,NaT,0,NaT,0,0,90
1,17195991,23542772,38319097,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2110-01-12 00:54:00,2110-01-14 22:07:39,2.884479,2110-01-11 22:47:00,2110-01-18 10:25:00,NaT,EW EMER.,EMERGENCY ROOM,CHRONIC/LONG TERM ACUTE CARE,Other,SINGLE,UNABLE TO OBTAIN,2110-01-01,NaT,F,0,0,0,0,1,1,0,0,NaT,NaT,0,NaT,0,0,90
2,16284044,23864737,36817845,Cardiac Vascular Intensive Care Unit (CVICU),Cardiac Vascular Intensive Care Unit (CVICU),2110-01-13 09:09:40,2110-01-15 21:45:33,2.524919,2110-01-11 19:58:00,2110-01-17 16:00:00,NaT,URGENT,TRANSFER FROM HOSPITAL,SKILLED NURSING FACILITY,Other,DIVORCED,WHITE,2110-01-01,NaT,F,0,0,0,0,1,1,0,0,NaT,NaT,0,NaT,0,0,90
3,18122666,23883987,39753527,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2110-01-14 07:22:00,2110-01-15 13:08:36,1.240694,2110-01-14 05:17:00,2110-01-16 16:34:00,NaT,EW EMER.,EMERGENCY ROOM,PSYCH FACILITY,Other,DIVORCED,WHITE,2110-01-01,NaT,M,0,0,0,0,1,1,0,0,NaT,NaT,0,NaT,0,0,90
4,18780420,28977824,32140208,Neuro Surgical Intensive Care Unit (Neuro SICU),Neuro Surgical Intensive Care Unit (Neuro SICU),2110-01-17 00:28:00,2110-01-18 01:21:22,1.037060,2110-01-16 23:37:00,2110-01-22 16:07:00,NaT,EW EMER.,EMERGENCY ROOM,REHAB,Medicare,MARRIED,WHITE,2110-01-01,NaT,M,0,0,0,0,1,1,0,0,NaT,NaT,0,NaT,0,0,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76535,15338454,29642380,37340595,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2210-11-21 18:42:56,2210-11-23 16:03:45,1.889456,2210-11-18 21:47:00,2210-12-01 17:51:00,NaT,EW EMER.,PHYSICIAN REFERRAL,HOSPICE,Medicare,SINGLE,WHITE,2200-01-01,2210-12-04,M,1,0,0,0,2,1,0,0,NaT,NaT,0,2 days 06:09:00,1,1,10
76536,18451791,28897092,32688519,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2211-01-05 10:59:50,2211-01-06 17:00:09,1.250220,2211-01-05 10:58:00,2211-01-07 16:30:00,NaT,DIRECT EMER.,PHYSICIAN REFERRAL,HOME,Other,MARRIED,OTHER,2206-01-01,NaT,M,0,0,0,0,1,1,0,0,NaT,NaT,0,NaT,0,0,5
76537,12844772,26783082,37001422,Neuro Surgical Intensive Care Unit (Neuro SICU),Neuro Surgical Intensive Care Unit (Neuro SICU),2211-01-14 14:37:00,2211-01-17 15:23:45,3.032465,2211-01-14 13:14:00,2211-01-17 12:34:00,2211-01-17 12:34:00,EW EMER.,PHYSICIAN REFERRAL,DIED,Medicare,SINGLE,WHITE,2199-01-01,2211-01-17,M,1,1,1,1,1,1,0,0,NaT,NaT,0,-1 days +11:26:00,0,2,12
76538,14239401,21199923,35906940,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2211-05-01 06:59:19,2211-05-10 22:51:06,9.660961,2211-05-01 06:57:00,2211-05-12 17:54:00,NaT,EW EMER.,PHYSICIAN REFERRAL,CHRONIC/LONG TERM ACUTE CARE,Medicare,WIDOWED,BLACK/AFRICAN AMERICAN,2200-01-01,NaT,F,0,0,0,0,1,1,0,0,NaT,NaT,0,NaT,0,0,11


In [36]:
stays['AGE'] = stays.apply(safe_age, axis=1).apply(lambda s: s / np.timedelta64(1, 's')) / 60./60/24/365
stays['AGE'] = stays['AGE'].astype('int64')
stays.loc[stays.AGE==0, 'AGE'] = 90
stays.AGE.value_counts()

90    46726
1      7785
2      4895
3      3864
4      3216
5      2378
6      2089
7      1635
8      1483
9      1116
10      822
11      426
12      105
Name: AGE, dtype: int64

In [43]:
prescriptions_dir = 'hosp/prescriptions.csv'

# drugs in this icu stay
drugs = pd.read_csv(os.path.join(MIMIC_PATH, prescriptions_dir))  # , index_col=0
drugs.columns = [x.upper() for x in drugs.columns]
drugs.head()
# icu_drugs = drugs.groupby(by='ICUSTAY_ID')['DRUG'].apply(list).reset_index()
# stays = stays.merge(icu_drugs[['ICUSTAY_ID', 'DRUG']], on='ICUSTAY_ID', how='left')

/tmp/ipykernel_18971/3712079728.py:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  drugs = pd.read_csv(os.path.join(MIMIC_PATH, prescriptions_dir))  # , index_col=0


,subject_id,hadm_id,pharmacy_id,starttime,stoptime,drug_type,drug,gsn,ndc,prod_strength,form_rx,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,doses_per_24_hrs,route
0,17868682,22726960,73313910,2160-01-07 08:00:00,2160-01-07 21:00:00,MAIN,BuPROPion (Sustained Release),046238,591083960.0,150mg SR Tablet,NaN,300,mg,2,TAB,1.0,PO
1,17868682,22726960,16239987,2160-01-07 08:00:00,2160-01-08 16:00:00,MAIN,BuPROPion (Sustained Release),046238,591083960.0,150mg SR Tablet,NaN,150,mg,1,TAB,1.0,PO
2,17868682,22726960,16634804,2160-01-07 10:00:00,2160-01-07 16:00:00,MAIN,Aspirin,004380,904404073.0,81mg Tab,NaN,81,mg,1,TAB,1.0,PO
3,17868682,22726960,2697460,2160-01-07 14:00:00,2160-01-07 14:00:00,BASE,1/2 NS,001209,338004304.0,1000mL Bag,NaN,1000,mL,1000,mL,NaN,IV
4,17868682,22726960,1383959,2160-01-07 15:00:00,2160-01-08 16:00:00,MAIN,Pneumococcal Vac Polyvalent,048548,6494300.0,25mcg/0.5mL Vial,NaN,0.5,mL,1,VIAL,NaN,IM


In [69]:
len(drugs)

17008053

In [70]:
# new_drugs_ = drugs.merge(icu_stays_simple, on=['SUBJECT_ID', 'HADM_ID'], how='left')
new_drugs = pd.merge(drugs, icu_stays_simple,  how='left', on=['SUBJECT_ID', 'HADM_ID'])
# icu_stays_simple = test_stays[['SUBJECT_ID', 'HADM_ID', 'STAY_ID']].copy()


In [71]:
len(new_drugs)

18712231

In [72]:
display(new_drugs)

,SUBJECT_ID,HADM_ID,PHARMACY_ID,STARTTIME,STOPTIME,DRUG_TYPE,DRUG,GSN,NDC,PROD_STRENGTH,FORM_RX,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,DOSES_PER_24_HRS,ROUTE,STAY_ID
0,17868682,22726960,73313910,2160-01-07 08:00:00,2160-01-07 21:00:00,MAIN,BuPROPion (Sustained Release),046238,5.910840e+08,150mg SR Tablet,NaN,300,mg,2,TAB,1.0,PO,NaN
1,17868682,22726960,16239987,2160-01-07 08:00:00,2160-01-08 16:00:00,MAIN,BuPROPion (Sustained Release),046238,5.910840e+08,150mg SR Tablet,NaN,150,mg,1,TAB,1.0,PO,NaN
2,17868682,22726960,16634804,2160-01-07 10:00:00,2160-01-07 16:00:00,MAIN,Aspirin,004380,9.044041e+08,81mg Tab,NaN,81,mg,1,TAB,1.0,PO,NaN
3,17868682,22726960,2697460,2160-01-07 14:00:00,2160-01-07 14:00:00,BASE,1/2 NS,001209,3.380043e+08,1000mL Bag,NaN,1000,mL,1000,mL,NaN,IV,NaN
4,17868682,22726960,1383959,2160-01-07 15:00:00,2160-01-08 16:00:00,MAIN,Pneumococcal Vac Polyvalent,048548,6.494300e+06,25mcg/0.5mL Vial,NaN,0.5,mL,1,VIAL,NaN,IM,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18712226,17608878,22135112,65762013,2147-04-12 17:00:00,2147-04-13 12:00:00,MAIN,Fentanyl Citrate,41384.0,1.747800e+10,100mcg/2mL Amp,NaN,100-200,mcg,2-4,mL,1.0,IV,37947672.0
18712227,15906963,20494713,9933337,2164-11-17 12:00:00,2164-11-18 11:00:00,MAIN,Fentanyl Citrate,41384.0,4.099093e+08,100mcg/2mL Amp,NaN,25-100,mcg,0.5-2,mL,1.0,IV,35296303.0
18712228,11667755,29607134,31327446,2126-08-08 05:00:00,2126-08-08 14:00:00,MAIN,OxycoDONE (Immediate Release),4225.0,4.060553e+08,5mg Tablet,NaN,7.5-10,mg,1-2,TAB,NaN,PO,NaN
18712229,17961555,22879995,50647416,2171-01-24 05:00:00,2171-02-03 20:00:00,MAIN,Acetaminophen-Caff-Butalbital,4451.0,5.913369e+08,1 Tablet,NaN,1-2,TAB,1-2,TAB,NaN,PO,32009796.0


In [73]:
# stays.dtypes
new_drugs.dtypes

SUBJECT_ID            int64
HADM_ID               int64
PHARMACY_ID           int64
STARTTIME            object
STOPTIME             object
DRUG_TYPE            object
DRUG                 object
GSN                  object
NDC                 float64
PROD_STRENGTH        object
FORM_RX              object
DOSE_VAL_RX          object
DOSE_UNIT_RX         object
FORM_VAL_DISP        object
FORM_UNIT_DISP       object
DOSES_PER_24_HRS    float64
ROUTE                object
STAY_ID             float64
dtype: object

In [75]:
new_drugs.rename(columns={'STAY_ID':'ICUSTAY_ID'}, inplace=True)
new_drugs['ICUSTAY_ID'] = new_drugs['ICUSTAY_ID'].astype('Int64')



In [77]:
icu_drugs = new_drugs.groupby(by='ICUSTAY_ID')['DRUG'].apply(list).reset_index()
display(icu_drugs)

,ICUSTAY_ID,DRUG
0,30000153,"[Topiramate (Topamax), Sodium Chloride 0.9% F..."
1,30000213,"[Propofol, Propofol, Soln, Fentanyl Citrate, S..."
2,30000484,"[Levothyroxine Sodium, Omeprazole, Vitamin D, ..."
3,30000646,"[Sodium Chloride 0.9% Flush, Enoxaparin Sodiu..."
4,30001148,"[Aspirin, Pravastatin, Lisinopril, Amlodipine,..."
...,...,...
76521,39999301,"[Pravastatin, Aspirin, NIFEdipine CR, Lisinopr..."
76522,39999384,"[Acetaminophen, Bisacodyl, Bisacodyl, HydrALAZ..."
76523,39999552,"[Sodium Chloride 0.9% Flush, Acetaminophen, F..."
76524,39999562,"[Acetaminophen, Gabapentin, Omeprazole, Atorva..."


In [80]:
stays.rename(columns={'STAY_ID':'ICUSTAY_ID'}, inplace=True)
print(len(stays))
display(stays)

76540


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,FIRST_CAREUNIT,LAST_CAREUNIT,INTIME,OUTTIME,LOS,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,MARITAL_STATUS,ETHNICITY,DOB,DOD,GENDER,MORTALITY,MORTALITY0,MORTALITY_INHOSPITAL,MORTALITY_INUNIT,COUNTS,MAX_OUTTIME,TRANSFERBACK,DIEINWARD,NEXT_INTIME,DIFF,LESS_THAN_30DAYS,DISCHARGE_DIE,DIE_LESS_THAN_30DAYS,READMISSION,AGE
0,18106347,24305596,30588857,Cardiac Vascular Intensive Care Unit (CVICU),Cardiac Vascular Intensive Care Unit (CVICU),2110-01-11 10:16:06,2110-01-12 17:17:47,1.292836,2110-01-11 10:14:00,2110-01-15 17:31:00,NaT,EW EMER.,PACU,HOME HEALTH CARE,Other,DIVORCED,WHITE,2110-01-01,NaT,F,0,0,0,0,1,1,0,0,NaT,NaT,0,NaT,0,0,90
1,17195991,23542772,38319097,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2110-01-12 00:54:00,2110-01-14 22:07:39,2.884479,2110-01-11 22:47:00,2110-01-18 10:25:00,NaT,EW EMER.,EMERGENCY ROOM,CHRONIC/LONG TERM ACUTE CARE,Other,SINGLE,UNABLE TO OBTAIN,2110-01-01,NaT,F,0,0,0,0,1,1,0,0,NaT,NaT,0,NaT,0,0,90
2,16284044,23864737,36817845,Cardiac Vascular Intensive Care Unit (CVICU),Cardiac Vascular Intensive Care Unit (CVICU),2110-01-13 09:09:40,2110-01-15 21:45:33,2.524919,2110-01-11 19:58:00,2110-01-17 16:00:00,NaT,URGENT,TRANSFER FROM HOSPITAL,SKILLED NURSING FACILITY,Other,DIVORCED,WHITE,2110-01-01,NaT,F,0,0,0,0,1,1,0,0,NaT,NaT,0,NaT,0,0,90
3,18122666,23883987,39753527,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2110-01-14 07:22:00,2110-01-15 13:08:36,1.240694,2110-01-14 05:17:00,2110-01-16 16:34:00,NaT,EW EMER.,EMERGENCY ROOM,PSYCH FACILITY,Other,DIVORCED,WHITE,2110-01-01,NaT,M,0,0,0,0,1,1,0,0,NaT,NaT,0,NaT,0,0,90
4,18780420,28977824,32140208,Neuro Surgical Intensive Care Unit (Neuro SICU),Neuro Surgical Intensive Care Unit (Neuro SICU),2110-01-17 00:28:00,2110-01-18 01:21:22,1.037060,2110-01-16 23:37:00,2110-01-22 16:07:00,NaT,EW EMER.,EMERGENCY ROOM,REHAB,Medicare,MARRIED,WHITE,2110-01-01,NaT,M,0,0,0,0,1,1,0,0,NaT,NaT,0,NaT,0,0,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76535,15338454,29642380,37340595,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2210-11-21 18:42:56,2210-11-23 16:03:45,1.889456,2210-11-18 21:47:00,2210-12-01 17:51:00,NaT,EW EMER.,PHYSICIAN REFERRAL,HOSPICE,Medicare,SINGLE,WHITE,2200-01-01,2210-12-04,M,1,0,0,0,2,1,0,0,NaT,NaT,0,2 days 06:09:00,1,1,10
76536,18451791,28897092,32688519,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2211-01-05 10:59:50,2211-01-06 17:00:09,1.250220,2211-01-05 10:58:00,2211-01-07 16:30:00,NaT,DIRECT EMER.,PHYSICIAN REFERRAL,HOME,Other,MARRIED,OTHER,2206-01-01,NaT,M,0,0,0,0,1,1,0,0,NaT,NaT,0,NaT,0,0,5
76537,12844772,26783082,37001422,Neuro Surgical Intensive Care Unit (Neuro SICU),Neuro Surgical Intensive Care Unit (Neuro SICU),2211-01-14 14:37:00,2211-01-17 15:23:45,3.032465,2211-01-14 13:14:00,2211-01-17 12:34:00,2211-01-17 12:34:00,EW EMER.,PHYSICIAN REFERRAL,DIED,Medicare,SINGLE,WHITE,2199-01-01,2211-01-17,M,1,1,1,1,1,1,0,0,NaT,NaT,0,-1 days +11:26:00,0,2,12
76538,14239401,21199923,35906940,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2211-05-01 06:59:19,2211-05-10 22:51:06,9.660961,2211-05-01 06:57:00,2211-05-12 17:54:00,NaT,EW EMER.,PHYSICIAN REFERRAL,CHRONIC/LONG TERM ACUTE CARE,Medicare,WIDOWED,BLACK/AFRICAN AMERICAN,2200-01-01,NaT,F,0,0,0,0,1,1,0,0,NaT,NaT,0,NaT,0,0,11


In [88]:
stays = stays.merge(icu_drugs[['ICUSTAY_ID', 'DRUG']], on='ICUSTAY_ID', how='left')
print(len(stays))
display(stays)

76540


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,FIRST_CAREUNIT,LAST_CAREUNIT,INTIME,OUTTIME,LOS,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,MARITAL_STATUS,ETHNICITY,DOB,DOD,GENDER,MORTALITY,MORTALITY0,MORTALITY_INHOSPITAL,MORTALITY_INUNIT,COUNTS,MAX_OUTTIME,TRANSFERBACK,DIEINWARD,NEXT_INTIME,DIFF,LESS_THAN_30DAYS,DISCHARGE_DIE,DIE_LESS_THAN_30DAYS,READMISSION,AGE,PROCEDURES,DRUG
0,18106347,24305596,30588857,Cardiac Vascular Intensive Care Unit (CVICU),Cardiac Vascular Intensive Care Unit (CVICU),2110-01-11 10:16:06,2110-01-12 17:17:47,1.292836,2110-01-11 10:14:00,2110-01-15 17:31:00,NaT,EW EMER.,PACU,HOME HEALTH CARE,Other,DIVORCED,WHITE,2110-01-01,NaT,F,0,0,0,0,1,1,0,0,NaT,NaT,0,NaT,0,0,90,"[9_3512, 9_3571, 9_3961]","[Sodium Chloride 0.9% Flush, Aspirin, Aspirin..."
1,17195991,23542772,38319097,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2110-01-12 00:54:00,2110-01-14 22:07:39,2.884479,2110-01-11 22:47:00,2110-01-18 10:25:00,NaT,EW EMER.,EMERGENCY ROOM,CHRONIC/LONG TERM ACUTE CARE,Other,SINGLE,UNABLE TO OBTAIN,2110-01-01,NaT,F,0,0,0,0,1,1,0,0,NaT,NaT,0,NaT,0,0,90,"[9_9671, 9_9604, 9_0331, 9_3324]","[Sodium Chloride 0.9% Flush, 0.9% Sodium Chlo..."
2,16284044,23864737,36817845,Cardiac Vascular Intensive Care Unit (CVICU),Cardiac Vascular Intensive Care Unit (CVICU),2110-01-13 09:09:40,2110-01-15 21:45:33,2.524919,2110-01-11 19:58:00,2110-01-17 16:00:00,NaT,URGENT,TRANSFER FROM HOSPITAL,SKILLED NURSING FACILITY,Other,DIVORCED,WHITE,2110-01-01,NaT,F,0,0,0,0,1,1,0,0,NaT,NaT,0,NaT,0,0,90,"[10_0210099, 10_021109W, 10_06BY4ZZ, 10_5A1221Z]","[Influenza Vaccine Quadrivalent, Acetaminophen..."
3,18122666,23883987,39753527,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2110-01-14 07:22:00,2110-01-15 13:08:36,1.240694,2110-01-14 05:17:00,2110-01-16 16:34:00,NaT,EW EMER.,EMERGENCY ROOM,PSYCH FACILITY,Other,DIVORCED,WHITE,2110-01-01,NaT,M,0,0,0,0,1,1,0,0,NaT,NaT,0,NaT,0,0,90,[9_4516],"[1/2 NS, Diazepam, Sodium Chloride 0.9% Flush..."
4,18780420,28977824,32140208,Neuro Surgical Intensive Care Unit (Neuro SICU),Neuro Surgical Intensive Care Unit (Neuro SICU),2110-01-17 00:28:00,2110-01-18 01:21:22,1.037060,2110-01-16 23:37:00,2110-01-22 16:07:00,NaT,EW EMER.,EMERGENCY ROOM,REHAB,Medicare,MARRIED,WHITE,2110-01-01,NaT,M,0,0,0,0,1,1,0,0,NaT,NaT,0,NaT,0,0,90,NaN,"[SW, 0.9% Sodium Chloride, PHENYLEPHrine, Hydr..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76535,15338454,29642380,37340595,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2210-11-21 18:42:56,2210-11-23 16:03:45,1.889456,2210-11-18 21:47:00,2210-12-01 17:51:00,NaT,EW EMER.,PHYSICIAN REFERRAL,HOSPICE,Medicare,SINGLE,WHITE,2200-01-01,2210-12-04,M,1,0,0,0,2,1,0,0,NaT,NaT,0,2 days 06:09:00,1,1,10,NaN,"[Lactated Ringers, Acetaminophen, Acetaminophe..."
76536,18451791,28897092,32688519,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2211-01-05 10:59:50,2211-01-06 17:00:09,1.250220,2211-01-05 10:58:00,2211-01-07 16:30:00,NaT,DIRECT EMER.,PHYSICIAN REFERRAL,HOME,Other,MARRIED,OTHER,2206-01-01,NaT,M,0,0,0,0,1,1,0,0,NaT,NaT,0,NaT,0,0,5,"[10_0W3P8ZZ, 10_0DB68ZZ, 10_0DJD8ZZ, 10_0DC68ZZ]","[Lactated Ringers, Ondansetron, Sodium Chlorid..."
76537,12844772,26783082,37001422,Neuro Surgical Intensive Care Unit (Neuro SICU),Neuro Surgical Intensive Care Unit (Neuro SICU),2211-01-14 14:37:00,2211-01-17 15:23:45,3.032465,2211-01-14 13:14:00,2211-01-17 12:34:00,2211-01-17 12:34:00,EW EMER.,PHYSICIAN REFERRAL,DIED,Medicare,SINGLE,WHITE,2199-01-01,2211-01-17,M,1,1,1,1,1,1,0,0,NaT,NaT,0,-1 days +11:26:00,0,2,12,"[10_03C80ZZ, 10_5A1945Z, 10_02HV33Z, 10_0B9F8ZX]","[Docusate Sodium, Bisacodyl, Bisacodyl, Acetam..."
76538,14239401,21199923,35906940,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2211-05

In [84]:
!ls data/mimic-iv/hosp/

d_hcpcs.csv.gz		 drgcodes.csv.gz	    poe.csv.gz
diagnoses_icd.csv	 emar.csv.gz		    poe_detail.csv.gz
diagnoses_icd.csv.gz	 emar_detail.csv.gz	    prescriptions.csv
d_icd_diagnoses.csv	 hcpcsevents.csv.gz	    prescriptions.csv.gz
d_icd_diagnoses.csv.gz	 labevents.csv.gz	    procedures_icd.csv
d_icd_procedures.csv.gz  microbiologyevents.csv.gz  procedures_icd.csv.gz
d_labitems.csv.gz	 pharmacy.csv.gz	    services.csv.gz


In [86]:
procedures_dir = 'hosp/procedures_icd.csv'
# procedures in this admission (even if not in this ICU stay)
procs = pd.read_csv(os.path.join(MIMIC_PATH, procedures_dir))  # , index_col=0

procs.columns = [x.upper() for x in procs.columns]
procs['ICD9_CODE'] = procs['ICD_VERSION'].astype(str) + '_' + procs['ICD_CODE'].astype(str)
print(procs.columns)
procs.drop('ICD_VERSION', axis=1, inplace=True)
procs.drop('ICD_CODE', axis=1, inplace=True)

display(procs)

Index(['SUBJECT_ID', 'HADM_ID', 'SEQ_NUM', 'CHARTDATE', 'ICD_CODE',
       'ICD_VERSION', 'ICD9_CODE'],
      dtype='object')


,SUBJECT_ID,HADM_ID,SEQ_NUM,CHARTDATE,ICD9_CODE
0,10287061,27485182,1,2124-08-22,9_7569
1,13135573,22286210,3,2187-05-30,9_7309
2,13135573,22286210,1,2187-05-30,9_7569
3,13135573,22286210,2,2187-05-30,9_734
4,12646116,24154012,2,2115-01-13,9_734
...,...,...,...,...,...
779620,13366767,26811822,1,2133-11-02,10_0VTTXZZ
779621,18980737,23514634,3,2146-06-28,10_0HQ9XZZ
779622,18980737,23514634,2,2146-06-28,10_10907ZC
779623,18980737,23514634,1,2146-06-28,10_10E0XZZ


In [87]:
admission_procs = procs.sort_values(by=['HADM_ID','SEQ_NUM']).groupby(by='HADM_ID')['ICD9_CODE'].apply(list).reset_index()
admission_procs = admission_procs.rename({'ICD9_CODE': 'PROCEDURES'}, axis=1)
stays = stays.merge(admission_procs[['HADM_ID', 'PROCEDURES']], on='HADM_ID', how='left')
display(stays)

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,FIRST_CAREUNIT,LAST_CAREUNIT,INTIME,OUTTIME,LOS,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,MARITAL_STATUS,ETHNICITY,DOB,DOD,GENDER,MORTALITY,MORTALITY0,MORTALITY_INHOSPITAL,MORTALITY_INUNIT,COUNTS,MAX_OUTTIME,TRANSFERBACK,DIEINWARD,NEXT_INTIME,DIFF,LESS_THAN_30DAYS,DISCHARGE_DIE,DIE_LESS_THAN_30DAYS,READMISSION,AGE,PROCEDURES
0,18106347,24305596,30588857,Cardiac Vascular Intensive Care Unit (CVICU),Cardiac Vascular Intensive Care Unit (CVICU),2110-01-11 10:16:06,2110-01-12 17:17:47,1.292836,2110-01-11 10:14:00,2110-01-15 17:31:00,NaT,EW EMER.,PACU,HOME HEALTH CARE,Other,DIVORCED,WHITE,2110-01-01,NaT,F,0,0,0,0,1,1,0,0,NaT,NaT,0,NaT,0,0,90,"[9_3512, 9_3571, 9_3961]"
1,17195991,23542772,38319097,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2110-01-12 00:54:00,2110-01-14 22:07:39,2.884479,2110-01-11 22:47:00,2110-01-18 10:25:00,NaT,EW EMER.,EMERGENCY ROOM,CHRONIC/LONG TERM ACUTE CARE,Other,SINGLE,UNABLE TO OBTAIN,2110-01-01,NaT,F,0,0,0,0,1,1,0,0,NaT,NaT,0,NaT,0,0,90,"[9_9671, 9_9604, 9_0331, 9_3324]"
2,16284044,23864737,36817845,Cardiac Vascular Intensive Care Unit (CVICU),Cardiac Vascular Intensive Care Unit (CVICU),2110-01-13 09:09:40,2110-01-15 21:45:33,2.524919,2110-01-11 19:58:00,2110-01-17 16:00:00,NaT,URGENT,TRANSFER FROM HOSPITAL,SKILLED NURSING FACILITY,Other,DIVORCED,WHITE,2110-01-01,NaT,F,0,0,0,0,1,1,0,0,NaT,NaT,0,NaT,0,0,90,"[10_0210099, 10_021109W, 10_06BY4ZZ, 10_5A1221Z]"
3,18122666,23883987,39753527,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2110-01-14 07:22:00,2110-01-15 13:08:36,1.240694,2110-01-14 05:17:00,2110-01-16 16:34:00,NaT,EW EMER.,EMERGENCY ROOM,PSYCH FACILITY,Other,DIVORCED,WHITE,2110-01-01,NaT,M,0,0,0,0,1,1,0,0,NaT,NaT,0,NaT,0,0,90,[9_4516]
4,18780420,28977824,32140208,Neuro Surgical Intensive Care Unit (Neuro SICU),Neuro Surgical Intensive Care Unit (Neuro SICU),2110-01-17 00:28:00,2110-01-18 01:21:22,1.037060,2110-01-16 23:37:00,2110-01-22 16:07:00,NaT,EW EMER.,EMERGENCY ROOM,REHAB,Medicare,MARRIED,WHITE,2110-01-01,NaT,M,0,0,0,0,1,1,0,0,NaT,NaT,0,NaT,0,0,90,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76535,15338454,29642380,37340595,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2210-11-21 18:42:56,2210-11-23 16:03:45,1.889456,2210-11-18 21:47:00,2210-12-01 17:51:00,NaT,EW EMER.,PHYSICIAN REFERRAL,HOSPICE,Medicare,SINGLE,WHITE,2200-01-01,2210-12-04,M,1,0,0,0,2,1,0,0,NaT,NaT,0,2 days 06:09:00,1,1,10,NaN
76536,18451791,28897092,32688519,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2211-01-05 10:59:50,2211-01-06 17:00:09,1.250220,2211-01-05 10:58:00,2211-01-07 16:30:00,NaT,DIRECT EMER.,PHYSICIAN REFERRAL,HOME,Other,MARRIED,OTHER,2206-01-01,NaT,M,0,0,0,0,1,1,0,0,NaT,NaT,0,NaT,0,0,5,"[10_0W3P8ZZ, 10_0DB68ZZ, 10_0DJD8ZZ, 10_0DC68ZZ]"
76537,12844772,26783082,37001422,Neuro Surgical Intensive Care Unit (Neuro SICU),Neuro Surgical Intensive Care Unit (Neuro SICU),2211-01-14 14:37:00,2211-01-17 15:23:45,3.032465,2211-01-14 13:14:00,2211-01-17 12:34:00,2211-01-17 12:34:00,EW EMER.,PHYSICIAN REFERRAL,DIED,Medicare,SINGLE,WHITE,2199-01-01,2211-01-17,M,1,1,1,1,1,1,0,0,NaT,NaT,0,-1 days +11:26:00,0,2,12,"[10_03C80ZZ, 10_5A1945Z, 10_02HV33Z, 10_0B9F8ZX]"
76538,14239401,21199923,35906940,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2211-05-01 06:59:19,2211-05-10 22:51:06,9.660961,2211-05-01 06:57:00,2211-05-12 17:54:00,NaT,EW EMER.,PHYSICIAN REFERRAL,CHRONIC/LONG TERM ACUTE CARE,Medicare,WIDOWED,BLACK/AFRICAN AMERICAN,2200-01-01,NaT,F,0,0,0,0,1,1,0,0,NaT,NaT,0,NaT,0,0,11,"[10_5A1955Z, 10_0BH17EZ, 10_3E0G76Z]"


In [90]:

# diags in this admission (even if not in this ICU stay)
# diag = pd.read_csv(os.path.join(MIMIC_PATH, 'DIAGNOSES_ICD.csv'), index_col=0)

diag = pd.read_csv(os.path.join(MIMIC_PATH, 'hosp/diagnoses_icd.csv'))
diag.columns = [x.upper() for x in diag.columns]
# diag.head()
diag['ICD9_CODE'] = diag['ICD_VERSION'].astype(str) + '_' + diag['ICD_CODE'].astype(str)
diag= diag[['SUBJECT_ID', 'HADM_ID', 'SEQ_NUM', 'ICD9_CODE']]


admission_diags = diag.sort_values(by=['HADM_ID','SEQ_NUM']).groupby(by='HADM_ID')['ICD9_CODE'].apply(list).reset_index()
admission_diags = admission_diags.rename({'ICD9_CODE': 'DIAGS'}, axis=1)

display(admission_diags)

,HADM_ID,DIAGS
0,20000019,"[9_0389, 9_59080, 9_75310, 9_5849, 9_2761, 9_9..."
1,20000024,"[10_D500, 10_K521, 10_I10, 10_E538, 10_M810, 1..."
2,20000034,"[10_K831, 10_K8689, 10_K861, 10_K869, 10_R1032..."
3,20000041,"[9_71536, 9_25002, 9_V8541, 9_4019, 9_2724, 9_..."
4,20000055,"[9_V3001, 9_V053]"
...,...,...
521106,29999745,"[9_29680, 9_29620, 9_2720, 9_3019, 9_V1541, 9_..."
521107,29999785,"[10_Z3801, 10_L22, 10_Z412, 10_Z00110, 10_Z23]"
521108,29999809,"[9_41401, 9_03842, 9_5990, 9_99591, 9_5961, 9_..."
521109,29999828,"[9_27801, 9_5180, 9_2762, 9_V8542, 9_4019, 9_2..."


In [91]:

# admis = pd.read_csv(os.path.join(MIMIC_PATH, 'ADMISSIONS.csv'), index_col=0)
admis = pd.read_csv(os.path.join(MIMIC_PATH, 'core/admissions.csv'))
admis.columns = [x.upper() for x in admis.columns]
admis.head()

admis = admis[['HADM_ID', 'SUBJECT_ID', 'ADMITTIME', 'DISCHTIME']]
admis.ADMITTIME = pd.to_datetime(admis.ADMITTIME)
admis.DISCHTIME = pd.to_datetime(admis.DISCHTIME)
admission_diags = admission_diags.merge(admis, on='HADM_ID')
admission_procs = admission_procs.merge(admis, on='HADM_ID')

In [92]:
# previous diags - from previous admissions
records = []

for i,r in tqdm(admission_diags.iterrows(), total=admission_diags.shape[0]):
    subj_id = r['SUBJECT_ID']
    in_time = r['ADMITTIME']
    prev_stays = admission_diags[(admission_diags.SUBJECT_ID == subj_id) & (admission_diags.DISCHTIME < in_time)]
    
    new_row = r
    new_row['NUM_PREV_ADMIS'] = len(prev_stays)
    if len(prev_stays) == 0:
        new_row['PREV_DIAGS'] = {}
        new_row['DAYS_SINCE_LAST_ADMIS'] = 0
    else:
        new_row['PREV_DIAGS'] = combine(prev_stays['DIAGS'].values)
        last_discharge = prev_stays['DISCHTIME'].max()
        new_row['DAYS_SINCE_LAST_ADMIS'] = (in_time - last_discharge)/np.timedelta64(1, 's')/60./60/24
    prev_procs = admission_procs[(admission_procs.SUBJECT_ID == subj_id) & (admission_procs.DISCHTIME < in_time)]
    new_row['NUM_PREV_PROCS'] = len(combine(prev_procs.PROCEDURES.values))
    records.append(new_row)
records = pd.DataFrame.from_records(records)

100%|██████████| 521110/521110 [1:10:14<00:00, 123.64it/s] 


In [93]:
display(records)

,HADM_ID,DIAGS,SUBJECT_ID,ADMITTIME,DISCHTIME,NUM_PREV_ADMIS,PREV_DIAGS,DAYS_SINCE_LAST_ADMIS,NUM_PREV_PROCS
0,20000019,"[9_0389, 9_59080, 9_75310, 9_5849, 9_2761, 9_9...",10467237,2159-03-20 21:08:00,2159-03-23 16:54:00,0,{},0.000000,0
1,20000024,"[10_D500, 10_K521, 10_I10, 10_E538, 10_M810, 1...",16925328,2151-05-25 21:44:00,2151-05-26 18:36:00,6,"[9_6820, 9_70703, 9_4019, 9_36250, 9_3659, 9_4...",24.165972,4
2,20000034,"[10_K831, 10_K8689, 10_K861, 10_K869, 10_R1032...",19430048,2174-05-22 19:13:00,2174-05-24 17:27:00,6,"[10_D62, 10_C3412, 10_N179, 10_N1330, 10_D538,...",55.165278,7
3,20000041,"[9_71536, 9_25002, 9_V8541, 9_4019, 9_2724, 9_...",18910522,2143-09-03 07:15:00,2143-09-06 13:50:00,1,"[9_71536, 9_2859, 9_2724, 9_32723, 9_25000, 9_...",402.489583,1
4,20000055,"[9_V3001, 9_V053]",11868001,2132-04-19 08:49:00,2132-04-23 11:30:00,0,{},0.000000,0
...,...,...,...,...,...,...,...,...,...
521105,29999745,"[9_29680, 9_29620, 9_2720, 9_3019, 9_V1541, 9_...",11326722,2160-07-05 22:24:00,2160-07-12 12:50:00,6,"[9_29680, 9_V6284, 9_30500, 9_311, 9_30981, 9_...",5.433333,4
521106,29999785,"[10_Z3801, 10_L22, 10_Z412, 10_Z00110, 10_Z23]",12012121,2110-08-08 00:00:00,2110-08-12 12:10:00,0,{},0.000000,0
521107,29999809,"[9_41401, 9_03842, 9_5990, 9_99591, 9_5961, 9_...",12133002,2172-09-24 21:27:00,2172-09-30 17:04:00,2,"[9_78659, 9_41401, 9_4019, 9_412, 9_V4582, 9_2...",1423.275694,5
521108,29999828,"[9_27801, 9_5180, 9_2762, 9_V8542, 9_4019, 9_2...",16572164,2180-10-27 07:15:00,2180-10-30 17:25:00,0,{},0.000000,0


In [108]:
display(stays)

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,FIRST_CAREUNIT,LAST_CAREUNIT,INTIME,OUTTIME,LOS,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,MARITAL_STATUS,ETHNICITY,DOB,DOD,GENDER,MORTALITY,MORTALITY0,MORTALITY_INHOSPITAL,MORTALITY_INUNIT,COUNTS,MAX_OUTTIME,TRANSFERBACK,DIEINWARD,NEXT_INTIME,DIFF,LESS_THAN_30DAYS,DISCHARGE_DIE,DIE_LESS_THAN_30DAYS,READMISSION,AGE,PROCEDURES,DRUG,DIAGS,PREV_DIAGS,DAYS_SINCE_LAST_ADMIS,NUM_PREV_ADMIS,NUM_PREV_PROCS
0,18106347,24305596,30588857,Cardiac Vascular Intensive Care Unit (CVICU),Cardiac Vascular Intensive Care Unit (CVICU),2110-01-11 10:16:06,2110-01-12 17:17:47,1.292836,2110-01-11 10:14:00,2110-01-15 17:31:00,NaT,EW EMER.,PACU,HOME HEALTH CARE,Other,DIVORCED,WHITE,2110-01-01,NaT,F,0,0,0,0,1,1,0,0,NaT,NaT,0,NaT,0,0,90,"[9_3512, 9_3571, 9_3961]","[Sodium Chloride 0.9% Flush, Aspirin, Aspirin...","[9_4240, 9_4295, 9_7455, 9_4019, 9_53081, 9_42...",{},0.000000,0.0,0.0
1,17195991,23542772,38319097,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2110-01-12 00:54:00,2110-01-14 22:07:39,2.884479,2110-01-11 22:47:00,2110-01-18 10:25:00,NaT,EW EMER.,EMERGENCY ROOM,CHRONIC/LONG TERM ACUTE CARE,Other,SINGLE,UNABLE TO OBTAIN,2110-01-01,NaT,F,0,0,0,0,1,1,0,0,NaT,NaT,0,NaT,0,0,90,"[9_9671, 9_9604, 9_0331, 9_3324]","[Sodium Chloride 0.9% Flush, 0.9% Sodium Chlo...","[9_3453, 9_5990, 9_29620, 9_2930, 9_70703, 9_7...",{},0.000000,0.0,0.0
2,16284044,23864737,36817845,Cardiac Vascular Intensive Care Unit (CVICU),Cardiac Vascular Intensive Care Unit (CVICU),2110-01-13 09:09:40,2110-01-15 21:45:33,2.524919,2110-01-11 19:58:00,2110-01-17 16:00:00,NaT,URGENT,TRANSFER FROM HOSPITAL,SKILLED NURSING FACILITY,Other,DIVORCED,WHITE,2110-01-01,NaT,F,0,0,0,0,1,1,0,0,NaT,NaT,0,NaT,0,0,90,"[10_0210099, 10_021109W, 10_06BY4ZZ, 10_5A1221Z]","[Influenza Vaccine Quadrivalent, Acetaminophen...","[10_I25118, 10_J95812, 10_I10, 10_D62, 10_J958...",{},0.000000,0.0,0.0
3,18122666,23883987,39753527,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2110-01-14 07:22:00,2110-01-15 13:08:36,1.240694,2110-01-14 05:17:00,2110-01-16 16:34:00,NaT,EW EMER.,EMERGENCY ROOM,PSYCH FACILITY,Other,DIVORCED,WHITE,2110-01-01,NaT,M,0,0,0,0,1,1,0,0,NaT,NaT,0,NaT,0,0,90,[9_4516],"[1/2 NS, Diazepam, Sodium Chloride 0.9% Flush...","[9_53140, 9_2920, 9_30471, 9_53021, 9_72210, 9...",{},0.000000,0.0,0.0
4,18780420,28977824,32140208,Neuro Surgical Intensive Care Unit (Neuro SICU),Neuro Surgical Intensive Care Unit (Neuro SICU),2110-01-17 00:28:00,2110-01-18 01:21:22,1.037060,2110-01-16 23:37:00,2110-01-22 16:07:00,NaT,EW EMER.,EMERGENCY ROOM,REHAB,Medicare,MARRIED,WHITE,2110-01-01,NaT,M,0,0,0,0,1,1,0,0,NaT,NaT,0,NaT,0,0,90,NaN,"[SW, 0.9% Sodium Chloride, PHENYLEPHrine, Hydr...","[10_I6319, 10_J690, 10_D696, 10_G8191, 10_I489...",{},0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76535,15338454,29642380,37340595,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2210-11-21 18:42:56,2210-11-23 16:03:45,1.889456,2210-11-18 21:47:00,2210-12-01 17:51:00,NaT,EW EMER.,PHYSICIAN REFERRAL,HOSPICE,Medicare,SINGLE,WHITE,2200-01-01,2210-12-04,M,1,0,0,0,2,1,0,0,NaT,NaT,0,2 days 06:09:00,1,1,10,NaN,"[Lactated Ringers, Acetaminophen, Acetaminophe...","[10_E1110, 10_I5023, 10_J9601, 10_J159, 10_N17...","[9_25080, 9_5859, 9_2767, 9_40390, 9_28529, 9_...",1412.393750,14.0,11.0
76536,18451791,28897092,32688519,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2211-01-05 10:59:50,2211-01-06 17:00:09,1.250220,2211-01-05 10:58:00,2211-01-07 16:30:00,NaT,DIRECT EMER.,PHYSICIAN REFERRAL,HOME,Other,MARRIED,OTHER,2206-01-01,NaT,M,0,0,0,0,1,1,0,0,NaT,NaT,0,NaT,0,0,5,"[10_0W3P8ZZ, 10_0DB68ZZ, 10_0DJD8ZZ, 10_0DC68ZZ]","[Lactated Ringers, Ondansetron, Sodium Chlorid...","[10_K

In [95]:
stays = stays.merge(records[['HADM_ID', 'DIAGS', 'PREV_DIAGS', 'DAYS_SINCE_LAST_ADMIS', 'NUM_PREV_ADMIS', 'NUM_PREV_PROCS']], on='HADM_ID', how='left')


# assignment = pd.read_csv(os.path.join(CUSTOM_TASKS_PATH, 'data', 'train_test_patients_full.csv'))
# stays = stays.merge(assignment, on='SUBJECT_ID')
# print(f"Divided stays to {len(stays[stays.ASSIGNMENT == 'train'])} train and {len(stays[stays.ASSIGNMENT == 'test'])} test stays")


stays.to_csv(os.path.join(CUSTOM_TASKS_PATH, 'data', 'stays_readmission_plus.csv'))

# sys.exit()


In [96]:
stays.READMISSION.value_counts()

0    58331
1    12940
2     5269
Name: READMISSION, dtype: int64

In [109]:
stays.columns

Index(['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'FIRST_CAREUNIT',
       'LAST_CAREUNIT', 'INTIME', 'OUTTIME', 'LOS', 'ADMITTIME', 'DISCHTIME',
       'DEATHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION',
       'DISCHARGE_LOCATION', 'INSURANCE', 'MARITAL_STATUS', 'ETHNICITY', 'DOB',
       'DOD', 'GENDER', 'MORTALITY', 'MORTALITY0', 'MORTALITY_INHOSPITAL',
       'MORTALITY_INUNIT', 'COUNTS', 'MAX_OUTTIME', 'TRANSFERBACK',
       'DIEINWARD', 'NEXT_INTIME', 'DIFF', 'LESS_THAN_30DAYS', 'DISCHARGE_DIE',
       'DIE_LESS_THAN_30DAYS', 'READMISSION', 'AGE', 'PROCEDURES', 'DRUG',
       'DIAGS', 'PREV_DIAGS', 'DAYS_SINCE_LAST_ADMIS', 'NUM_PREV_ADMIS',
       'NUM_PREV_PROCS'],
      dtype='object')

In [110]:
stays.dtypes

SUBJECT_ID                         int64
HADM_ID                            int64
ICUSTAY_ID                         int64
FIRST_CAREUNIT                    object
LAST_CAREUNIT                     object
INTIME                    datetime64[ns]
OUTTIME                   datetime64[ns]
LOS                              float64
ADMITTIME                 datetime64[ns]
DISCHTIME                 datetime64[ns]
DEATHTIME                 datetime64[ns]
ADMISSION_TYPE                    object
ADMISSION_LOCATION                object
DISCHARGE_LOCATION                object
INSURANCE                         object
MARITAL_STATUS                    object
ETHNICITY                         object
DOB                       datetime64[ns]
DOD                       datetime64[ns]
GENDER                            object
MORTALITY                          int64
MORTALITY0                         int64
MORTALITY_INHOSPITAL               int64
MORTALITY_INUNIT                   int64
COUNTS          

In [ ]:
UNIQUE_IDS =['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID']
BINARY_CATEGORIES = ['MORTALITY', 'MORTALITY0','MORTALITY_INHOSPITAL', 'MORTALITY_INUNIT','TRANSFERBACK',
       'DIEINWARD', 'LESS_THAN_30DAYS', 'DIE_LESS_THAN_30DAYS']  
CATEGORIES = [
    'FIRST_CAREUNIT', 'LAST_CAREUNIT',  'ADMISSION_TYPE', 'ADMISSION_LOCATION',
       'DISCHARGE_LOCATION', 'INSURANCE', 'MARITAL_STATUS', 'ETHNICITY', 
        'GENDER',  'COUNTS', 'MAX_OUTTIME',   'AGE']
DATES_CAT = ['INTIME', 'OUTTIME', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME', 'DOB', 'DOD', 'NEXT_INTIME',]
DURATION_CAT = ['DIFF', 'DISCHARGE_DIE', ]
NUMERIC_CAT =['DAYS_SINCE_LAST_ADMIS', 'NUM_PREV_ADMIS', 'NUM_PREV_PROCS']
MULTIPlE_STR_EMBED = ['PROCEDURES', 'DRUG', 'DIAGS', 'PREV_DIAGS',]

LABELS_CAT = ['LOS', 'READMISSION']

```html
For numeric features, I need to scale them using standardScaler; for categorical ones, perform one-hot encoding and use countvectorizer for the text columns.

Transformer{‘drop’, ‘passthrough’}, or estimator

The estimator must support fit and transform. I would be using StandardScaler for numeric, CountVectorizer for unstructured text column, and OneHotEncoder for the categorical column.

For my dataset, I need to pass the below-mentioned transformers in the parameter of the transformers of the ColumnTransfomer function.

SimpleImputer with constant imputing to categorical columns [‘Division Name’,’Department Name’]
CountVectorizer with STOPWORDS on ‘Review Text’ and just CountVectorizer on the ‘Title’ column.
SimpleImputer with median imputing for numerical columns [‘Clothing ID’,’Age’,’Rating’,‘Recommended IND’,‘Positive Feedback Count’].

STOPWORDS = set(stopwords.words('english'))

catTransformer = Pipeline(steps=[('cat_imputer', SimpleImputer(strategy='constant', fill_value='missing')),('cat_ohe', OneHotEncoder(handle_unknown='ignore'))])

textTransformer_0 = Pipeline(steps=[
    ('text_bow', CountVectorizer(lowercase=True,\
                                 token_pattern=r"(?u)\b\w+\b",\
                                 stop_words=STOPWORDS))])

textTransformer_1 = Pipeline(steps=[('text_bow1', CountVectorizer())])

numeric_features = ['Clothing ID','Age','Rating','Recommended IND','Positive Feedback Count']

numTransformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),('scaler', StandardScaler())])

ct = ColumnTransformer(
transformers=[
('cat', catTransformer, ['Division Name','Department Name']),
('num', numTransformer, numeric_features),
('text1', textTransformer_0, 'Review Text'),
('text2', textTransformer_1, 'Title')
])

```

In [121]:
stays.NUM_PREV_PROCS.value_counts()

0.0      44763
2.0       3773
1.0       3750
3.0       2919
4.0       2557
         ...  
131.0        1
136.0        1
142.0        1
143.0        1
111.0        1
Name: NUM_PREV_PROCS, Length: 121, dtype: int64

In [98]:
stays.FIRST_CAREUNIT.value_counts()

Medical Intensive Care Unit (MICU)                  16729
Medical/Surgical Intensive Care Unit (MICU/SICU)    13421
Cardiac Vascular Intensive Care Unit (CVICU)        12169
Surgical Intensive Care Unit (SICU)                 11765
Trauma SICU (TSICU)                                  9165
Coronary Care Unit (CCU)                             8746
Neuro Surgical Intensive Care Unit (Neuro SICU)      1851
Neuro Intermediate                                   1823
Neuro Stepdown                                        871
Name: FIRST_CAREUNIT, dtype: int64

In [99]:
stays.ADMISSION_TYPE.value_counts()

EW EMER.                       40721
URGENT                         13084
OBSERVATION ADMIT               9410
SURGICAL SAME DAY ADMISSION     7356
DIRECT EMER.                    2843
ELECTIVE                        2522
EU OBSERVATION                   404
DIRECT OBSERVATION               177
AMBULATORY OBSERVATION            23
Name: ADMISSION_TYPE, dtype: int64

In [100]:
stays.GENDER.value_counts()

M    42755
F    33785
Name: GENDER, dtype: int64

In [101]:
stays.ETHNICITY.value_counts()

WHITE                            51801
BLACK/AFRICAN AMERICAN            8338
UNKNOWN                           6722
OTHER                             3495
HISPANIC/LATINO                   2891
ASIAN                             2257
UNABLE TO OBTAIN                   884
AMERICAN INDIAN/ALASKA NATIVE      152
Name: ETHNICITY, dtype: int64

In [102]:
stays.INSURANCE.value_counts()

Other       36098
Medicare    34602
Medicaid     5840
Name: INSURANCE, dtype: int64

In [103]:
stays.DISCHARGE_LOCATION.value_counts()

HOME HEALTH CARE                18972
HOME                            17947
SKILLED NURSING FACILITY        14114
DIED                             8795
REHAB                            6180
CHRONIC/LONG TERM ACUTE CARE     5605
HOSPICE                          1679
PSYCH FACILITY                    803
AGAINST ADVICE                    680
ACUTE HOSPITAL                    670
OTHER FACILITY                    325
ASSISTED LIVING                    92
HEALTHCARE FACILITY                17
Name: DISCHARGE_LOCATION, dtype: int64

In [ ]:
# read ICU stays
stays = pd.read_csv(os.path.join(MIMIC_PATH, 'ICUSTAYS.csv'), index_col=0)
stays.INTIME = pd.to_datetime(stays.INTIME)

admis = pd.read_csv(os.path.join(MIMIC_PATH, 'ADMISSIONS.csv'), index_col=0)
admis.DISCHTIME = pd.to_datetime(admis.DISCHTIME)
admis.ADMITTIME = pd.to_datetime(admis.ADMITTIME)
admis = admis.sort_values('ADMITTIME', ascending=True)
admis_gb = admis.groupby('SUBJECT_ID')
records = []
for subj_id, group in admis_gb:
    if len(group) < 2:
        continue
    else:
        prev_visit_end = None
        prev_visit_start = None
        prev_adm_id = None
        prev_ethnic = None
        
        for i, admission_row in group.iterrows():
            if prev_visit_end is None:
                prev_visit_end = admission_row['DISCHTIME']
                prev_visit_start = admission_row['ADMITTIME']
                prev_adm_id = admission_row['HADM_ID']
                prev_ethnic = admission_row['ETHNICITY']
            else:
                record = {'SUBJECT_ID': subj_id,
                          'ETHNICITY':  prev_ethnic,
                          'HADM_ID': prev_adm_id,
                          'ADMITTIME': prev_visit_start,
                          'DISCHTIME': prev_visit_end,
                          'NEXT_ADMITTIME': admission_row['ADMITTIME']}
                records.append(record)
                prev_visit_end = admission_row['DISCHTIME']
                prev_visit_start = admission_row['ADMITTIME']
                prev_adm_id = admission_row['HADM_ID']
                prev_ethnic = admission_row['ETHNICITY']

admissions = pd.DataFrame.from_records(records)
# We count the number of days to the next admission.
admissions['TIME_TO_READMISSION'] = (admissions['NEXT_ADMITTIME'] - admissions['DISCHTIME']).apply(lambda s: s / np.timedelta64(1, 's')) / 60./60/24
# and convert to a binary label.
admissions['READMITTED30'] = admissions['TIME_TO_READMISSION'] < 30


# take all the diagnoses and group them by admission
diag = pd.read_csv(os.path.join(MIMIC_PATH, 'DIAGNOSES_ICD.csv'), index_col=0)
admission_diags = diag.sort_values(by=['HADM_ID','SEQ_NUM']).groupby(by='HADM_ID')['ICD9_CODE'].apply(list).reset_index()
admission_diags = admission_diags.rename({'ICD9_CODE': 'DIAGS'}, axis=1)
admissions = admissions.merge(admission_diags[['HADM_ID', 'DIAGS']], on='HADM_ID', how='left')
del admission_diags, diag

# same for procedures
procs = pd.read_csv(os.path.join(MIMIC_PATH, 'PROCEDURES_ICD.csv'), index_col=0)
admission_procs = procs.sort_values(by=['HADM_ID','SEQ_NUM']).groupby(by='HADM_ID')['ICD9_CODE'].apply(list).reset_index()
admission_procs = admission_procs.rename({'ICD9_CODE': 'PROCEDURES'}, axis=1)
admissions = admissions.merge(admission_procs[['HADM_ID', 'PROCEDURES']], on='HADM_ID', how='left')
del admission_procs, procs

# and the same for drugs
drugs = pd.read_csv(os.path.join(MIMIC_PATH, 'PRESCRIPTIONS.csv'), index_col=0)
admis_drugs = drugs.groupby(by='HADM_ID')['DRUG'].apply(set).reset_index()
admissions = admissions.merge(admis_drugs[['HADM_ID', 'DRUG']], on='HADM_ID', how='left')
del drugs, admis_drugs

print(f"found {len(admissions)} records who had previous hospital visits from {admissions.SUBJECT_ID.nunique()} unique subjects.")

# get demographics from ADMISSIONS and gender from PATIENTS
patients = pd.read_csv(os.path.join(MIMIC_PATH, 'PATIENTS.csv'), index_col=0)
patients.DOB = pd.to_datetime(patients.DOB)

admissions = admissions.merge(patients[['SUBJECT_ID', 'GENDER', 'DOB']], on='SUBJECT_ID')
admissions['AGE'] = admissions.apply(safe_age, axis=1).apply(lambda s: s / np.timedelta64(1, 's')) / 60./60/24/365
admissions.loc[admissions.AGE.isna(), 'AGE'] = 90

# assign to train and test (0.15)
#TODO: use the same assignment from the los_by_diag task!!
subject_assignment = pd.read_csv(os.path.join(CUSTOM_TASKS_PATH, 'data', 'train_test_patients.csv'), index_col=0)['ASSIGNMENT'].to_dict()
admissions['ASSIGNMENT'] = admissions['SUBJECT_ID'].apply(lambda x: subject_assignment[x])

print(f"Train-test division: {admissions.ASSIGNMENT.value_counts()}")
admissions.to_csv(os.path.join(CUSTOM_TASKS_PATH, 'data', 'readmission_by_diag_plus.csv'))

In [123]:
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import Normalizer
ct = ColumnTransformer(
    [("norm1", Normalizer(norm='l1'), [0, 1]),
     ("norm2", Normalizer(norm='l1'), slice(2, 4))])
X = np.array([[0., 1., 2., 2.],
              [1., 1., 0., 1.]])
# Normalizer scales each row of X to unit norm. A separate scaling
# is applied for the two first and two last elements of each
# row independently.
ct.fit_transform(X)

array([[0. , 1. , 0.5, 0.5],
       [0.5, 0.5, 0. , 1. ]])

In [124]:
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import MinMaxScaler
import pandas as pd   
X = pd.DataFrame({
    "documents": ["First item", "second one here", "Is this the last?"],
    "width": [3, 4, 5],
})  
# "documents" is a string which configures ColumnTransformer to
# pass the documents column as a 1d array to the FeatureHasher
ct = ColumnTransformer(
    [("text_preprocess", FeatureHasher(input_type="string"), "documents"),
     ("num_preprocess", MinMaxScaler(), ["width"])])
X_trans = ct.fit_transform(X)  

In [125]:
X_trans

<3x1048577 sparse matrix of type '<class 'numpy.float64'>'
	with 29 stored elements in Compressed Sparse Row format>